In [2]:
from Feeds.infura_multichain import Web3Infura, get_checksum_address
from Feeds.etherscan import EtherscanClient
from Feeds.utility import *

from Feeds.coingecko import *
from Feeds.dexscreener import *

from datetime import datetime
import warnings
import urllib3
import pandas as pd


# Suppress only the InsecureRequestWarning
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

# Skrypt do szukania wspólnych adresów
Okeślamy liczbe projektów oraz daty w których sprawdzamy listę holderów, następnie sprawdzamy i analizujemy wspólnych holderów
Dodatkowo możemy sprawdzić wartość bieżącą portfeli lub wartość poszczególnych projektów, aby odsiać małe pozycje. Raczej portfele z małymi pozycjami nie 
będą stanowiły dobrego adresu do naśladowania.

In [3]:
ETHERSCAN_API_KEY = 'KR52G3163AU9696NCAZ74CGFKJZV7GIIG6'


addresses = {
    "sample_random_0": "0x0f000bFDD83597758011189898073653d97bf557",
    "sample_random_1": "0x5664773F216D8006D5cfEc4c9De324395d05b422",
    "sample_random_2": "0x1929d4d2b476f38ad06688c4b11fd01aacf4c9f5",
    "NeuralAI": "0x32B053F2CBA79F80ada5078cb6b305da92BDe6e1",
    "SharpAI": "0xaddb6dc7e2f7caea67621dd3ca2e8321ade33286",
    "INFRA": "0xe9EccDE9d26FCBB5e93F536CFC4510A7f46274f8",
    "KNDX": "0x7CA5af5bA3472AF6049F63c1AbC324475D44EFC1",
    "BUILD": "0x73454ACfdDb7a36A3cd8Eb171fBEa86c6a55E550",
    "TAOBOT": "0x49fb8ad7578148E17c3eF0C344CE23A66ed372C4",
    "Vmint": "0xD7B2C1a7F3c67fB0EA57a7ef29bC1F18D7bE3195",
    "CHAT": "0xBb3D7F42C58Abd83616Ad7C8C72473Ee46df2678",
}

In [4]:
def get_common_holderss(tokens_with_dates):
    """
    Find common holders across multiple tokens based on 'to' addresses.
    tokens_with_dates: List of tuples [(token_contract, datetime), ...]
    """
    cls = EtherscanClient(api_key=ETHERSCAN_API_KEY)
    all_holders_df = None  # Initialize as None to hold the first DataFrame

    for token_contract, date in tokens_with_dates:
        if date:
            # Convert date to UNIX timestamp and get block number
            timestamp = datetime_to_unix_timestamp(date)
            block_number = cls.get_block_number_by_timestamp(timestamp)
            print(f"Fetching holders for token {token_contract} at block {block_number}")
            holders_df = cls.get_token_transfers_by_address(contract_address=token_contract, start_block=0, end_block=block_number)
        else:
            # Fetch current holders
            print(f"Fetching current holders for token {token_contract}")
            holders_df = cls.get_token_transfers_by_address(contract_address=token_contract)

        # Filter out only the 'to' column from the holders DataFrame
        holders_df = holders_df[['to']].drop_duplicates()

        # If this is the first token, initialize the DataFrame
        if all_holders_df is None:
            all_holders_df = holders_df
        else:
            # Merge the current holders with the previous ones to find common addresses
            all_holders_df = pd.merge(all_holders_df, holders_df, on='to', how='inner')

    # Return the final DataFrame with common holders across all tokens
    return all_holders_df

In [5]:
# Example usage
tokens_with_dates = [
    ("0x3567aa22cd3ab9aEf23d7e18EE0D7cf16974d7e6", datetime(2024, 10, 15)),  # Sharp AI
    # ("0x32B053F2CBA79F80ada5078cb6b305da92BDe6e1", None),  # Neural AI
    # ("0x7CA5af5bA3472AF6049F63c1AbC324475D44EFC1", datetime(2024, 10, 19)),  # KNDX (Kondux)
    # ("0x73454ACfdDb7a36A3cd8Eb171fBEa86c6a55E550", datetime(2024, 2, 24)),  # BUILD (Build AI)
    # ("0x49fb8ad7578148E17c3eF0C344CE23A66ed372C4", None),  # TAOBOT (Tao.Bot)
    # ("0xe9EccDE9d26FCBB5e93F536CFC4510A7f46274f8", None),  # INFRA (Infra)
    # ("0xD7B2C1a7F3c67fB0EA57a7ef29bC1F18D7bE3195", None),  # Vmint (VoluMint)
    ("0xBb3D7F42C58Abd83616Ad7C8C72473Ee46df2678", None),  # CHAT (Vector Chat)
]

common_holders = get_common_holderss(tokens_with_dates)
print(f"Common holders: {len(common_holders)} \n{common_holders['to'].to_list()}")

Fetching holders for token 0x3567aa22cd3ab9aEf23d7e18EE0D7cf16974d7e6 at block 20966646
Fetching blockchain data from block 1...
last_block: 20723886, Num trans: 10000
Fetching blockchain data from block 20723887...
last_block: 20966308, Num trans: 4650
Fetching current holders for token 0xBb3D7F42C58Abd83616Ad7C8C72473Ee46df2678
Fetching blockchain data from block 1...
last_block: 18946351, Num trans: 10000
Fetching blockchain data from block 18946352...
last_block: 19305705, Num trans: 10000
Fetching blockchain data from block 19305706...
last_block: 19799805, Num trans: 10000
Fetching blockchain data from block 19799806...
last_block: 20842958, Num trans: 10000
Fetching blockchain data from block 20842959...
last_block: 21043330, Num trans: 4463
Common holders: 55 
['0xfeb37531c44701c48fa9dd4256dc0f97c9568cb9', '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad', '0x000000fee13a103a10d593b9ae06b3e05f2e7e1c', '0x22f9dcf4647084d6c31b2765f6910cd85c178c18', '0x382ffce2287252f930e1c8dc9328dac5b